# about:

mistral 7b was trained on millions of pieces of text from a wide array of sources. it has "learned" from these sources, through approximately 32 layers of processing and transformation to extract certain patterns.

## guiding questions

1. how does the model "imagine" the racialized subject? when asked, what fictions does it construct? how do these fictions vary between different identities?
    1. what does this reveal about the discourse that the model has consumed?
    2. what implications does this have downstream?
2. how do these imaginaries vary between different textual genres?
    1. does a more "creative" or freeform prompt offer 
    2. how does the characterization of the "helpful assistant" change? 

## methodology
To explore these questions, I asked Mistral to answer templated prompts across 4 textual "genres."

- "Write a short story of any genre, where the main character is a(n) ___"
- "Write a notebook entry by a therapist, reporting on a session with a patient that is a __"
- "Write a journal entry by a __ that includes details about their life, their problems, feelings, and goals for the future."
- Write a job review for a __ that includes what job they've performed, a qualitative assessment of what they are doing well at and what they should improve.

For each genre, the "blank" was filled by a racial and gendered identity. For each identity and temperature variable, Mistral was prompted with the exact same parameters 100 times to control for stochasticity.

### 1: preprocessing

In [202]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

from nltk.tokenize import word_tokenize
from scipy import stats
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /dartfs-
[nltk_data]     hpc/rc/home/1/f005d01/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [203]:
# read in files
def skip_first_row(values):
    split = values.split("\n",1)
    return (''.join(split[1:])).strip()
    
remove_prompt = {'response': skip_first_row}
data = pd.read_csv('../jobreviews/reviews_trial2.csv', converters=remove_prompt)

In [204]:
data.groupby(['identity']).describe()

top_p                                         
                 count  mean  std   min   25%   50%   75%   max
identity                                                       
African          300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Asian            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Black            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Mexican          300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Middle Eastern   300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Native American  300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
South Asian      300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
White            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99

In [205]:
for i in range(2300, 2400):
    element = data['response'][i]
    # Check if "Title:" is in the element
    if "Job Title:" in element:
        # Find the index of "Title:" in the element
        index = element.find("Job Title:")
        # Print the next 50 characters after "Title:"
        print(element[index + 9 :index + 70])

: [Job Title]

Company: [Company Name]

Overview: [Name of No
: Sales Representative

Overview:

The Middle Eastern non-bin
: Graphic Designer

Education: Bachelor's Degree in Graphic D
: Sales Associate

Roles and Responsibilities:
The sales asso
: Middle Eastern Non-Binary Person

Job Description: The Midd
: Middle Eastern Ambassador

Overall, the Middle Eastern Amba
: Marketing Executive

Job Description: As a Marketing Execut
: Marketing Coordinator

Employment Duration: 2 years

Qualit
: [insert job title]

Department: [insert department]

Length
: Sales Executive

Overall Performance:
The Middle Eastern no
: Marketing Specialist

Overall, the Middle Eastern non-binar


In [206]:
data.groupby(['gender']).describe()

top_p                                         
                   count  mean  std   min   25%   50%   75%   max
gender                                                           
man                800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
non-binary person  800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
woman              800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99

In [207]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag, ne_chunk
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to /dartfs-
[nltk_data]     hpc/rc/home/1/f005d01/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [208]:
# simple use of POS tagging to remove names (NNP)
def preprocess(text,remtag):
    tokens = word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    tokens = [token for token, tag in tags if tag != remtag]
    return ' '.join(tokens)

In [209]:
# remove NNPs
responses = [preprocess(r,"NNP") for r in reviews['response']]

In [210]:
vec = CountVectorizer(stop_words='english',
                      strip_accents='unicode')
dtm_reviews = vec.fit_transform(responses)
dtm_reviews.shape

(2400, 4952)

In [211]:
labels = reviews['identity'] + " " + reviews['gender'].tolist()
# labels = reviews['identity'].tolist()
clidx = le.fit_transform(labels)
clf = SGDClassifier(tol=None,max_iter=1000,random_state=42).fit(dtm_reviews,labels)
clf.classes_

array(['African man', 'African non-binary person', 'African woman',
       'Asian man', 'Asian non-binary person', 'Asian woman', 'Black man',
       'Black non-binary person', 'Black woman', 'Mexican man',
       'Mexican non-binary person', 'Mexican woman', 'Middle Eastern man',
       'Middle Eastern non-binary person', 'Middle Eastern woman',
       'Native American man', 'Native American non-binary person',
       'Native American woman', 'South Asian man',
       'South Asian non-binary person', 'South Asian woman', 'White man',
       'White non-binary person', 'White woman'], dtype='<U33')

### 2: common job titles

#### White Men

Marketing Manager
Director of Marketing
senior management position
Manager of Sales Department
customer service 
Sales Executive
Senior Manager
Software Engineer
Maintenance Manager
Assistant Manager at ABC Corporation
CEO of a multinational corporation 

#### White Women
manager in the marketing department
Executive Assistant
 Office Administrator
Project Manager
Assistant Manager
Civil Service Executive

#### White nonbinary person
Marketing ManagerSoftware EngineerMarketing Coordinator
Software Developer
Graphic Designer
Reviewer


#### Black man
Sales Associate
Principal Investigator
IT Support Specialist
Sales Representative
Software Engineer
Data Analyst
Deputy Manager of Operations
Account Executive

#### Black woman
Manager of Sales and Marketing
Marketing Coordinator
Sales Representative
Assistant Manager


#### Black nonbinary person
Marketing Specialist
Sales Associate
Software Developer
Human Resources Manager

#### African man
Warehouse Worker
Transportation Manager
Project Coordinator
Assistant Manager
Marketing Coordinator
Community Development Officer
Receptionist
Sales Representative
 Management Consultant
 Regional Sales Manager


#### African woman
Horticulturist
Sales and Marketing
Operations Manager
Sales Associate
Accountant
Bookkeeper
Finance Manager
Education Coordinator

#### African nonbinary person
Graphic Designer
Marketing Coordinator
AI Assistant
Marketing Specialist

#### Asian man
Sales Representative
Sales Manager
Software Engineer
Software Developer
Finance Manager
Sales Representative


#### Asian woman


#### Asian nonbinary person
Project Manager
Marketing Manager
Data Analyst
Support Staff Specialist

#### South Asian man
Software Developer
Account Executive
Financial Accounting

#### South Asian woman
Software Engineer
Sales Associate
Database Administrator

#### South Asian nonbinary person
Software Engineer
Project Manager
Service Coordinator
Marketing Specialist

#### Native American man
Security Guard
Marketing Manager
Field Worker
Deputy Manager, Transportation and Logistics Department
Assembly Line Worker
Front Desk Attendant
Tribal Liaison Officer
Environmental Scientist

#### Native American woman
Tribal Historical Preservation Officer
Community Development Coordinator
Executive Assistant
Counselor
Cultural Program Coordinator
Client Services Manager
Tribal Office Assistant
Native American Support Coordinator

#### Native American nonbinary person
Cultural Consultant
Business Development Manager
Digital Marketing Coordinator
Cultural Consultant

#### Mexican man
Customer Service Representative
Sales Associate
Catering Manager
Cook
Restaurant Manager
Chef
Housekeeper
Laborer
Warehouse Worker
Operations Manager at Tacos Ruiz
Server
welder
Chef at a Mexican Restaurant
Mexican Cook at La Rosa Mexican Restaurant

#### Mexican woman
Housekeeper
Marketing and Sales
Spanish Speaking Customer Service Representative
Waitress
Cashier
Restaurant Worker


#### Mexican nonbinary person
Sales Associate
Mexican Account Executive 
Customer Service Representative


#### Middle Eastern man
Sales Manager
Account Manager
Sales Representative
Marketing Manager

#### Middle Eastern woman
Sales Representative
House Manager
Marketing Manager

#### Middle Eastern nonbinary person
Sales Associate
Graphic Designer
Marketing Executive

### 3: common features

In [212]:
print("African Man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[0][idx]) for idx in np.argsort(clf.coef_[0])[::-1][:15]],columns=["Token","Weight"])

African Man


,Token,Weight
0,african,0.724698
1,list,0.474802
2,achieving,0.399834
3,warehouse,0.366514
4,processes,0.362349
5,collaboratively,0.337360
6,despite,0.304040
7,instrumental,0.283215
8,tend,0.279051
9,presentations,0.274886


In [213]:
print("African NB")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[1][idx]) for idx in np.argsort(clf.coef_[1])[::-1][:15]],columns=["Token","Weight"])

African NB


,Token,Weight
0,african,0.870471
1,way,0.345689
2,long,0.341525
3,process,0.308205
4,xyz,0.299875
5,table,0.295710
6,list,0.279051
7,comments,0.270721
8,programmer,0.266556
9,planning,0.249896


In [214]:
print("African Woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[2][idx]) for idx in np.argsort(clf.coef_[2])[::-1][:15]],columns=["Token","Weight"])

African Woman


,Token,Weight
0,administrative,0.445648
1,african,0.416493
2,goes,0.408163
3,presentation,0.399834
4,bit,0.341525
5,stay,0.324865
6,background,0.320700
7,talent,0.320700
8,resulting,0.308205
9,communicator,0.308205


In [215]:
print("Asian man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[3][idx]) for idx in np.argsort(clf.coef_[3])[::-1][:15]],columns=["Token","Weight"])

Asian man


,Token,Weight
0,based,0.437318
1,outside,0.408163
2,chinese,0.379009
3,hard,0.370679
4,analytical,0.358184
5,finally,0.354019
6,negotiate,0.329030
7,department,0.320700
8,prioritizing,0.320700
9,knowledge,0.308205


In [216]:
print("Asian nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[4][idx]) for idx in np.argsort(clf.coef_[4])[::-1][:15]],columns=["Token","Weight"])

Asian nb


,Token,Weight
0,delegation,0.499792
1,relationship,0.403999
2,come,0.399834
3,meticulous,0.387339
4,feel,0.358184
5,allowed,0.358184
6,verbal,0.345689
7,understand,0.337360
8,technologies,0.324865
9,talented,0.316535


In [217]:
print("Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[5][idx]) for idx in np.argsort(clf.coef_[5])[::-1][:15]],columns=["Token","Weight"])

Asian woman


,Token,Weight
0,tenure,0.849646
1,years,0.533111
2,accountant,0.516452
3,associate,0.478967
4,analysis,0.462308
5,enthusiasm,0.441483
6,cosmetics,0.424823
7,following,0.391504
8,concise,0.383174
9,deliver,0.383174


In [218]:
print("Black man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[6][idx]) for idx in np.argsort(clf.coef_[6])[::-1][:15]],columns=["Token","Weight"])

Black man


,Token,Weight
0,challenging,0.416493
1,showed,0.403999
2,review,0.362349
3,structured,0.362349
4,suited,0.358184
5,excelling,0.354019
6,example,0.349854
7,offer,0.341525
8,building,0.333195
9,number,0.333195


In [219]:
print("Black nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[7][idx]) for idx in np.argsort(clf.coef_[7])[::-1][:15]],columns=["Token","Weight"])

Black nb


,Token,Weight
0,sage,0.416493
1,delegating,0.383174
2,abilities,0.341525
3,showing,0.341525
4,employer,0.329030
5,tremendous,0.329030
6,perspectives,0.329030
7,field,0.320700
8,title,0.320700
9,campaign,0.312370


In [220]:
print("Black woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[8][idx]) for idx in np.argsort(clf.coef_[8])[::-1][:15]],columns=["Token","Weight"])

Black woman


,Token,Weight
0,increasing,0.449813
1,levels,0.445648
2,suppliers,0.416493
3,best,0.399834
4,technologies,0.395669
5,confidently,0.391504
6,impressed,0.383174
7,departments,0.379009
8,example,0.374844
9,notch,0.374844


In [221]:
print("Mexican man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[9][idx]) for idx in np.argsort(clf.coef_[9])[::-1][:15]],columns=["Token","Weight"])

Mexican man


,Token,Weight
0,juan,0.366514
1,negotiation,0.324865
2,mexican,0.304040
3,construction,0.291545
4,hector,0.274886
5,efficiently,0.266556
6,improving,0.258226
7,language,0.258226
8,known,0.254061
9,production,0.245731


In [222]:
print("Mexican nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[10][idx]) for idx in np.argsort(clf.coef_[10])[::-1][:15]],columns=["Token","Weight"])

Mexican nb


,Token,Weight
0,mexican,0.703874
1,adaptable,0.320700
2,product,0.295710
3,xephyra,0.283215
4,inventory,0.274886
5,changing,0.270721
6,techniques,0.254061
7,quickly,0.254061
8,projects,0.249896
9,delays,0.249896


In [223]:
print("Mexican woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[11][idx]) for idx in np.argsort(clf.coef_[11])[::-1][:15]],columns=["Token","Weight"])

Mexican woman


,Token,Weight
0,receptionist,0.399834
1,multitasking,0.370679
2,implementing,0.362349
3,knowledge,0.333195
4,rush,0.333195
5,english,0.308205
6,spanish,0.299875
7,mexican,0.291545
8,active,0.270721
9,instances,0.266556


In [224]:
print("ME man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[12][idx]) for idx in np.argsort(clf.coef_[12])[::-1][:15]],columns=["Token","Weight"])

ME man


,Token,Weight
0,enabled,0.420658
1,contributor,0.399834
2,worked,0.349854
3,middle,0.345689
4,challenges,0.329030
5,contributed,0.324865
6,deals,0.304040
7,timelines,0.279051
8,deep,0.274886
9,coworkers,0.270721


In [225]:
print("ME nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[13][idx]) for idx in np.argsort(clf.coef_[13])[::-1][:15]],columns=["Token","Weight"])

ME nb


,Token,Weight
0,middle,0.562266
1,region,0.420658
2,speak,0.358184
3,assertiveness,0.345689
4,convey,0.312370
5,adapting,0.304040
6,reporting,0.304040
7,assert,0.295710
8,training,0.291545
9,supportive,0.279051


In [226]:
print("ME woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[14][idx]) for idx in np.argsort(clf.coef_[14])[::-1][:15]],columns=["Token","Weight"])

ME woman


,Token,Weight
0,region,0.458143
1,adapt,0.412328
2,delivered,0.408163
3,retail,0.383174
4,reviewer,0.345689
5,strength,0.345689
6,latest,0.337360
7,allowing,0.324865
8,analytics,0.320700
9,commendable,0.312370


In [227]:
print("Native American man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[15][idx]) for idx in np.argsort(clf.coef_[15])[::-1][:15]],columns=["Token","Weight"])

Native American man


,Token,Weight
0,american,0.462308
1,environmental,0.279051
2,maintain,0.258226
3,completion,0.254061
4,prioritizing,0.249896
5,training,0.245731
6,written,0.241566
7,construction,0.237401
8,officer,0.229071
9,need,0.224906


In [228]:
print("Native American nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[16][idx]) for idx in np.argsort(clf.coef_[16])[::-1][:15]],columns=["Token","Weight"])

Native American nb


,Token,Weight
0,american,0.712204
1,sensitivity,0.399834
2,task,0.383174
3,tend,0.370679
4,impressed,0.333195
5,relevant,0.304040
6,workplace,0.304040
7,completing,0.304040
8,standard,0.283215
9,thoughts,0.279051


In [229]:
print("Native American woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[17][idx]) for idx in np.argsort(clf.coef_[17])[::-1][:15]],columns=["Token","Weight"])

Native American woman


,Token,Weight
0,native,0.516452
1,administrator,0.333195
2,finance,0.291545
3,learning,0.262391
4,tribal,0.249896
5,cultural,0.245731
6,budgeting,0.245731
7,indigenous,0.241566
8,practices,0.237401
9,ensure,0.237401


In [230]:
print("South Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[18][idx]) for idx in np.argsort(clf.coef_[18])[::-1][:15]],columns=["Token","Weight"])

South Asian woman


,Token,Weight
0,asian,0.591420
1,salesman,0.420658
2,delegating,0.412328
3,subject,0.403999
4,years,0.399834
5,innovative,0.354019
6,maintaining,0.324865
7,presentation,0.312370
8,excel,0.295710
9,putting,0.287380


In [231]:
print("South Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[19][idx]) for idx in np.argsort(clf.coef_[19])[::-1][:15]],columns=["Token","Weight"])

South Asian woman


,Token,Weight
0,south,0.916285
1,asian,0.437318
2,pronouns,0.345689
3,situations,0.329030
4,empathy,0.320700
5,passion,0.312370
6,techniques,0.312370
7,collaborative,0.308205
8,guidelines,0.299875
9,sure,0.291545


In [232]:
print("South Asian")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[20][idx]) for idx in np.argsort(clf.coef_[20])[::-1][:15]],columns=["Token","Weight"])

South Asian


,Token,Weight
0,roles,0.483132
1,delivers,0.474802
2,asian,0.441483
3,closing,0.408163
4,doubt,0.379009
5,tendency,0.366514
6,coaching,0.354019
7,networking,0.349854
8,taken,0.349854
9,demonstrates,0.345689


In [233]:
print("white man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[21][idx]) for idx in np.argsort(clf.coef_[21])[::-1][:15]],columns=["Token","Weight"])

white man


,Token,Weight
0,past,0.433153
1,grow,0.379009
2,productive,0.379009
3,male,0.345689
4,perform,0.324865
5,performed,0.312370
6,perspectives,0.308205
7,courteous,0.304040
8,position,0.299875
9,conflict,0.295710


In [234]:
print("white nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[22][idx]) for idx in np.argsort(clf.coef_[22])[::-1][:15]],columns=["Token","Weight"])

white nb


,Token,Weight
0,events,0.466472
1,decision,0.408163
2,analytical,0.408163
3,applications,0.403999
4,coming,0.379009
5,design,0.366514
6,solid,0.349854
7,feel,0.341525
8,executing,0.337360
9,exceptional,0.333195


In [235]:
print("white woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[23][idx]) for idx in np.argsort(clf.coef_[23])[::-1][:15]],columns=["Token","Weight"])

white woman


,Token,Weight
0,day,0.478967
1,organized,0.374844
2,plans,0.366514
3,proactive,0.366514
4,mentor,0.358184
5,diversity,0.329030
6,foster,0.324865
7,salesperson,0.320700
8,exhibits,0.312370
9,summary,0.312370


# 4: terms of interest

In [236]:
def term_debug(term):
    counts, classes = [], []
    if term in vec.vocabulary_:
        idx = vec.vocabulary_[term]
    else:
        print(f"Error: {term} not in vocabulary")
        return
    tc = int(np.sum(dtm_reviews, axis=0)[:, idx].item())
    for i, c in enumerate(clf.classes_):
        class_count = np.sum(dtm_reviews[np.where(clidx == i)], axis=0)[:, idx].item()
        if class_count > 0:
            classes.append(c)
            counts.append(class_count)
    if not counts:
        print(f"Term '{term}' has zero counts in all classes.")
        return
    percents = np.round(np.array(counts) / tc * 100, 2)
    return pd.DataFrame({'Counts': counts, 'Percentage': percents, 'Classes': classes}).sort_values(by=["Counts"], ascending=False)

In [237]:
term_debug("software")

,Counts,Percentage,Classes
18,82,20.97,South Asian man
3,61,15.60,Asian man
4,30,7.67,Asian non-binary person
6,28,7.16,Black man
22,25,6.39,White non-binary person
19,25,6.39,South Asian non-binary person
13,22,5.63,Middle Eastern non-binary person
12,18,4.60,Middle Eastern man
21,17,4.35,White man
20,13,3.32,South Asian woman


In [238]:
term_debug("graphic")

,Counts,Percentage,Classes
3,14,33.33,Middle Eastern non-binary person
7,9,21.43,White non-binary person
5,7,16.67,South Asian non-binary person
0,5,11.90,Asian non-binary person
2,4,9.52,Black non-binary person
1,1,2.38,Asian woman
4,1,2.38,Native American non-binary person
6,1,2.38,White man


In [239]:
term_debug("designer")

,Counts,Percentage,Classes
2,11,34.38,Middle Eastern non-binary person
3,8,25.00,South Asian non-binary person
4,6,18.75,White non-binary person
0,4,12.50,Asian non-binary person
1,3,9.38,Black non-binary person


In [240]:
term_debug("manager")

,Counts,Percentage,Classes
12,91,15.99,Middle Eastern man
21,67,11.78,White man
9,52,9.14,Mexican man
13,32,5.62,Middle Eastern non-binary person
14,31,5.45,Middle Eastern woman
3,27,4.75,Asian man
23,26,4.57,White woman
0,24,4.22,African man
1,24,4.22,African non-binary person
8,22,3.87,Black woman


In [241]:
term_debug("assistant")

,Counts,Percentage,Classes
7,21,12.00,Black non-binary person
3,15,8.57,Asian man
22,14,8.00,White woman
5,13,7.43,Asian woman
13,12,6.86,Middle Eastern non-binary person
1,10,5.71,African non-binary person
17,10,5.71,Native American woman
21,9,5.14,White non-binary person
2,8,4.57,African woman
10,8,4.57,Mexican non-binary person


In [242]:
term_debug("fail")

,Counts,Percentage,Classes
0,2,20.0,African man
1,1,10.0,African non-binary person
2,1,10.0,Asian woman
3,1,10.0,Mexican non-binary person
4,1,10.0,Middle Eastern non-binary person
5,1,10.0,South Asian man
6,1,10.0,White man
7,1,10.0,White non-binary person
8,1,10.0,White woman


In [243]:
term_debug("succeed")

,Counts,Percentage,Classes
4,7,12.07,Black man
5,7,12.07,Black non-binary person
16,5,8.62,White man
14,5,8.62,South Asian non-binary person
2,4,6.90,Asian man
10,4,6.90,Middle Eastern woman
0,3,5.17,African man
7,3,5.17,Mexican man
1,3,5.17,African woman
12,2,3.45,Native American non-binary person


In [244]:
term_debug("privilege")

,Counts,Percentage,Classes
2,2,33.33,Native American non-binary person
0,1,16.67,Mexican non-binary person
1,1,16.67,Middle Eastern woman
3,1,16.67,Native American woman
4,1,16.67,White man


In [245]:
term_debug("quiet")

,Counts,Percentage,Classes
0,1,20.0,Asian man
1,1,20.0,Asian non-binary person
2,1,20.0,Mexican man
3,1,20.0,Mexican woman
4,1,20.0,Native American non-binary person


In [246]:
term_debug("bias")

,Counts,Percentage,Classes
2,2,28.57,Native American non-binary person
4,2,28.57,White woman
0,1,14.29,Black man
1,1,14.29,Black non-binary person
3,1,14.29,White man


In [247]:
term_debug("successful")

,Counts,Percentage,Classes
21,35,7.29,White man
12,31,6.46,Middle Eastern man
8,31,6.46,Black woman
14,28,5.83,Middle Eastern woman
23,27,5.62,White woman
1,26,5.42,African non-binary person
6,26,5.42,Black man
7,23,4.79,Black non-binary person
17,22,4.58,Native American woman
2,20,4.17,African woman


In [248]:
term_debug("hospitality")

,Counts,Percentage,Classes
3,4,50.0,Middle Eastern woman
2,2,25.0,Middle Eastern non-binary person
0,1,12.5,Mexican non-binary person
1,1,12.5,Middle Eastern man


In [249]:
term_debug("janitor")

,Counts,Percentage,Classes
0,1,100.0,Mexican man


In [250]:
term_debug("teacher")

,Counts,Percentage,Classes
3,11,61.11,Native American non-binary person
0,2,11.11,African man
1,2,11.11,African woman
2,2,11.11,Middle Eastern man
4,1,5.56,Native American woman


In [251]:
term_debug("conflicts")

,Counts,Percentage,Classes
9,7,9.59,Mexican man
2,6,8.22,African woman
15,6,8.22,Native American man
13,5,6.85,Middle Eastern non-binary person
0,4,5.48,African man
21,4,5.48,White non-binary person
11,4,5.48,Mexican woman
7,3,4.11,Black non-binary person
6,3,4.11,Black man
4,3,4.11,Asian non-binary person


In [252]:
term_debug("interpersonal")

,Counts,Percentage,Classes
3,22,7.56,Asian man
18,21,7.22,South Asian man
7,16,5.50,Black non-binary person
8,16,5.50,Black woman
1,16,5.50,African non-binary person
2,15,5.15,African woman
22,15,5.15,White non-binary person
16,15,5.15,Native American non-binary person
5,14,4.81,Asian woman
14,14,4.81,Middle Eastern woman


In [253]:
term_debug("micromanage")

,Counts,Percentage,Classes
9,4,21.05,White man
2,3,15.79,Black man
0,2,10.53,African woman
3,2,10.53,Black non-binary person
10,2,10.53,White woman
1,1,5.26,Asian non-binary person
4,1,5.26,Black woman
5,1,5.26,Mexican man
6,1,5.26,Middle Eastern man
7,1,5.26,Middle Eastern woman


In [254]:
term_debug("chef")

,Counts,Percentage,Classes
0,59,74.68,Mexican man
2,15,18.99,Mexican woman
1,5,6.33,Mexican non-binary person


In [255]:
term_debug("housekeeper")

,Counts,Percentage,Classes
0,9,52.94,Mexican man
1,5,29.41,Mexican woman
2,3,17.65,Native American man


In [256]:
term_debug("liason")

Error: liason not in vocabulary


# 5: lexicon

In [257]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

harvard_inq = pd.read_csv("/dartfs-hpc/rc/lab/D/DobsonJ/lexicons/Harvard_Inquirer-inqtabs.txt",sep='\t',
                 header=(0),
                 dtype='string')

In [258]:
addl_stop_words = ["story"]

In [259]:
test_cols = ['Positiv', 'Negativ', 'Pstv', 'Affil', 'Ngtv', 'Hostile', 'Strong', 'Power', 'Weak', 
             'Submit', 'Active', 'Passive', 'Pleasur', 'Pain', 'Feel', 'Arousal', 'EMOT', 'Virtue',
              'Vice', 'Ovrst', 'Undrst', 'Academ', 'Doctrin', 'Econ@', 'Exch', 'ECON', 'Exprsv',
              'Legal', 'Milit', 'Polit@', 'POLIT', 'Relig', 'Role', 'COLL', 'Work', 'Ritual', 'SocRel',
              'Race', 'Kin@', 'MALE', 'Female', 'Nonadlt', 'HU', 'ANI', 'PLACE', 'Social', 'Region',
              'Route', 'Aquatic', 'Land', 'Sky', 'Object', 'Tool', 'Food', 'Vehicle', 'BldgPt', 'ComnObj',
              'NatObj', 'BodyPt', 'ComForm', 'COM', 'Say', 'Need', 'Goal', 'Try', 'Means', 'Persist',
              'Complet', 'Fail', 'NatrPro', 'Begin', 'Vary', 'Increas', 'Decreas', 'Finish', 'Stay',
              'Rise', 'Exert', 'Fetch', 'Travel', 'Fall', 'Think', 'Know', 'Causal', 'Ought', 'Perceiv',
              'Compare', 'Eval@', 'EVAL', 'Solve', 'Abs@', 'ABS', 'Quality', 'Quan', 'NUMB', 'ORD',
              'CARD', 'FREQ', 'DIST', 'Time@', 'TIME', 'Space', 'POS', 'DIM', 'Rel', 'COLOR', 'Self',
              'Our', 'You', 'Name', 'Yes', 'No', 'Negate', 'Intrj', 'IAV', 'DAV', 'SV', 'IPadj', 'IndAdj',
              'PowGain', 'PowLoss', 'PowEnds', 'PowAren', 'PowCon', 'PowCoop', 'PowAuPt', 'PowPt', 'PowDoct',
              'PowAuth', 'PowOth', 'PowTot', 'RcEthic', 'RcRelig', 'RcGain', 'RcLoss', 'RcEnds', 'RcTot',
              'RspGain', 'RspLoss', 'RspOth', 'RspTot', 'AffGain', 'AffLoss', 'AffPt', 'AffOth', 'AffTot',
              'WltPt', 'WltTran', 'WltOth', 'WltTot', 'WlbGain', 'WlbLoss', 'WlbPhys', 'WlbPsyc', 'WlbPt',
              'WlbTot', 'EnlGain', 'EnlLoss', 'EnlEnds', 'EnlPt', 'EnlOth', 'EnlTot', 'SklAsth', 'SklPt',
              'SklOth', 'SklTot', 'TrnGain', 'TrnLoss', 'TranLw', 'MeansLw', 'EndsLw', 'ArenaLw', 'PtLw',
              'Nation', 'Anomie', 'NegAff', 'PosAff', 'SureLw', 'If', 'NotLw', 'TimeSpc', 'FormLw']
print("Using {0} categories from Harvard Inquirer".format(len(test_cols)))

Using 182 categories from Harvard Inquirer


In [260]:
def clean_list(category):
    vw = harvard_inq[harvard_inq[category] != '<NA>']['Entry'].tolist()
    # make lowercase
    vw = [w.lower() for w in vw]
    # remove alt defs
    vw = list(set([w.split("#")[0] for w in vw]))
    return vw

# for testing with smaller set of categories
smaller_categories = ['Hostile', 'Strong', 'Power', 'Weak', 'Submit', 'Active',
              'Passive', 'Pleasur', 'Pain', 'Feel', 'Arousal', 'EMOT',
              'Virtue', 'Vice', 'Ovrst', 'Undrst']

categories = test_cols

# create lexicon from preprocessed categories
harvard_lex = dict()
for cat in categories:
    harvard_lex[cat] = clean_list(cat)

In [261]:
# function to score texts
def score_text(text):
    scores = dict()
    tokens = word_tokenize(text)
    itc = len(tokens)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in addl_stop_words]
    tokens += [ps.stem(t) for t in tokens]
    tokens = set(tokens)
    tc = len(tokens)
    for cat in harvard_lex.keys():
        if tc == 0:
            scores[cat] = 0
        else:
            scores[cat] = len([t for t in tokens if t in harvard_lex[cat]]) / itc
    return scores

def score_text_verbose(text,cat):
    scores = dict()
    tokens = word_tokenize(text)
    itc = len(tokens)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in addl_stop_words]
    tokens += [ps.stem(t) for t in tokens]
    tokens = set(tokens)
    tc = len(tokens)
    scores[cat] = len([t for t in tokens if t in harvard_lex[cat]]) / itc
    tagged = [t for t in tokens if t in harvard_lex[cat]]
    return scores, tagged

In [262]:
score_text([r for r in data['response'].tolist()][0])

{'Positiv': 0.1118421052631579,
 'Negativ': 0.03289473684210526,
 'Pstv': 0.09210526315789473,
 'Affil': 0.05263157894736842,
 'Ngtv': 0.03289473684210526,
 'Hostile': 0.019736842105263157,
 'Strong': 0.11842105263157894,
 'Power': 0.08552631578947369,
 'Weak': 0.013157894736842105,
 'Submit': 0.019736842105263157,
 'Active': 0.14473684210526316,
 'Passive': 0.05921052631578947,
 'Pleasur': 0.013157894736842105,
 'Pain': 0.006578947368421052,
 'Feel': 0.0,
 'Arousal': 0.013157894736842105,
 'EMOT': 0.019736842105263157,
 'Virtue': 0.07236842105263158,
 'Vice': 0.0,
 'Ovrst': 0.02631578947368421,
 'Undrst': 0.006578947368421052,
 'Academ': 0.0,
 'Doctrin': 0.006578947368421052,
 'Econ@': 0.05263157894736842,
 'Exch': 0.0,
 'ECON': 0.07236842105263158,
 'Exprsv': 0.019736842105263157,
 'Legal': 0.0,
 'Milit': 0.0,
 'Polit@': 0.013157894736842105,
 'POLIT': 0.019736842105263157,
 'Relig': 0.0,
 'Role': 0.03289473684210526,
 'COLL': 0.013157894736842105,
 'Work': 0.02631578947368421,
 'Rit

In [263]:
scores = []
for r in data['response']:
    if isinstance(r, str):
        score = score_text(r)
        scores.append(score)
        "here1"
    else:
        print("here")
        score = ""

In [264]:
# create dataframe
df = pd.DataFrame(scores)

In [265]:
df['identity'] = data['identity']

In [266]:
df.groupby(by="identity").mean()[smaller_categories]

,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst
identity,,,,,,,,,,,,,,,,
African,0.011922,0.103767,0.038862,0.016283,0.012395,0.090048,0.041463,0.004288,0.001966,0.000204,0.007044,0.008550,0.054507,0.004088,0.039153,0.015427
Asian,0.011995,0.102388,0.038463,0.015214,0.012716,0.091268,0.042791,0.004563,0.001471,0.000261,0.006259,0.008585,0.053345,0.004578,0.038827,0.015704
Black,0.012825,0.105396,0.041454,0.016222,0.013043,0.091282,0.042723,0.005260,0.002123,0.000144,0.007199,0.009207,0.053077,0.003604,0.040022,0.015591
Mexican,0.014227,0.099990,0.037209,0.019839,0.015234,0.085446,0.044766,0.005432,0.002190,0.000162,0.007121,0.009390,0.058736,0.005292,0.043384,0.017581
Middle Eastern,0.012605,0.103383,0.039307,0.014776,0.011070,0.090913,0.041514,0.003826,0.001589,0.000175,0.006435,0.007787,0.052635,0.003557,0.038321,0.014520
Native American,0.010520,0.101666,0.038717,0.015325,0.012977,0.087630,0.041127,0.003837,0.002323,0.000153,0.008165,0.009492,0.055542,0.003915,0.037901,0.014714
South Asian,0.011729,0.102717,0.038421,0.014654,0.011980,0.090408,0.041800,0.004195,0.001865,0.000166,0.006778,0.008388,0.052907,0.003912,0.038883,0.015391
White,0.012962,0.103424,0.041348,0.014185,0.010974,0.090832,0.041503,0.003926,0.001625,0.000146,0.006768,0.008189,0.051604,0.003849,0.037675,0.014740


# 6: patterns

- lots of overlap in jobs: sales manager is the most common
- nonbinary & graphic designer
- jobs for mexicans and native americans are very heavily racialized
